In [1]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.6 MB/s eta 0:00:00


In [2]:
import numpy as np
from Bio.Align import substitution_matrices
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

/usr/local/lib/python3.10/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


#Жадный алгоритм множественного выравнивания (7 баллов)
 Реализуйте алгоритм, который принимал бы на вход массив строк, штраф за удаления, вставки и несовпадения, а так-же цену совпадений. А возвращал бы множественное выравнивание. На первом шаге алгоритм должен выбрать две самые близкие по расстоянию Левенштейна строки и заменить их консенснусной строкой. При следующих шагах алгоритма выравниваться между собой могут так же и консенснусные строки. При этом стоит хранить для каждой строки не только ее саму но и профиль множественного выравнивания, чтобы в итоге правильно пересчитывать консенсус.
Результат работы алгоритма - массив строк, соответствующий некоторому множественному выравниванию.

In [3]:
matrix = substitution_matrices.load('BLOSUM62')

In [14]:
def levenshtein_distance( shot_s, long_s, delet, mattch, mismatch):
    n, m = len(long_s), len(shot_s)
    ins = delet
    dist = [[i*ins for i in range(m+1)], [0]*(m+1)]
    for i in range(1, n+1):
        dist[1][0] = i*delet
        for j in range(1, m+1):
          if shot_s[j-1] == long_s[i-1]:
            dist[1][j] = max(dist[0][j-1] + mattch, dist[0][j] +delet, dist[1][j-1]+ins)
          else:
            dist[1][j] = max(dist[0][j-1] + mismatch, dist[0][j] + delet, dist[1][j-1] + ins)
        dist[0] = dist[1][:]
    return dist[1][-1]

In [26]:
def calculate_consensus(profs):
    #Рассчет консенсусной строки по сумме профилей
    #Считаем что в консенсусе не должно быть пробелов
    bases = np.array(['A', 'T', 'G', 'C', '-'])
    consensus = ''.join(list(bases[np.argmax(profs[:,:4], axis=1)]))
    return consensus

In [27]:
def get_prof(seq, old_seq, ind, profs):
    #Пересчет профиля с учетов гэпов в выравнивании на консенсус
    #берем старый и добавляем гэпы веса равного количеству строк этого консенсуса
    old_prof = profs[ind]
    w = np.sum(old_prof[0])
    new_prof = np.zeros((len(seq), 5))
    i, j = 0, 0
    while i<len(seq) or j<len(old_seq):
        if j < len(old_seq) and seq[i] == old_seq[j]:
          new_prof[i] = old_prof[j]
          j += 1
        else:
          new_prof[i][-1] = w
        i += 1
    return new_prof

In [28]:
def Needleman_Wunsch(s1, s2, matrix, gap_w=-1):
    bases = {'A':0, 'T':1, 'G':2, 'C':3, '-':4}
    n, m = len(s1), len(s2)
    dp = np.vstack([np.array([0+i*gap_w for i in range(n+1)]), np.zeros((m, n+1))])
    vector = np.zeros((m+1, n+1))
    vector[:, 0] = 1
    vector[0] = 2
    for i in range(1, m+1):
      dp[i, 0] = dp[i-1, 0] + gap_w
      for j in range(1, n+1):
          a, b = bases[s2[i-1]], bases[s1[j-1]]
          dp[i][j] = max(dp[i-1, j] + gap_w,
                         dp[i, j-1] + gap_w,
                         dp[i-1, j-1] + matrix[a][b])
          if dp[i, j] == dp[i-1, j] + gap_w:
                vector[i, j] = 1  # up
          elif dp[i, j] == dp[i, j-1] + gap_w:
                vector[i, j] = 2  # left
          else:
                vector[i, j] = 0  # giag


    aligned_s1 = []
    aligned_s2 = []
    view = []
    i, j = m, n
    while i > 0 or j > 0:
        if vector[i, j] == 0:
            aligned_s2.append(s2[i-1])
            aligned_s1.append(s1[j-1])
            if s2[i-1] == s1[j-1]:
                view += '|'
            else:
                view += '.'
            i -= 1
            j -= 1
        elif vector[i, j] == 1:
            aligned_s2.append(s2[i-1])
            aligned_s1.append('-')
            view += ' '
            i -= 1
        else:
            aligned_s2.append('-')
            aligned_s1.append(s1[j-1])
            view += ' '
            j -= 1
    aligned_s1.reverse()
    aligned_s2.reverse()
    return [aligned_s1, aligned_s2]

In [29]:
def first(seq):
  #Задание весовых матриц для всех последовательностей
  bases = {'A':0, 'T':1, 'G':2, 'C':3, '-':4}
  prof = np.zeros((len(seq), 5))
  for i in range(len(seq)):
    pos = bases[seq[i]]
    prof[i][pos] = 1
  return prof

In [30]:
def get_string(strings, alg, scoring_matrix):
    #Вывод результатов выравнивания всех последовательностей на итоговый консенсус
    print('conc:', alg)
    for s in strings:
      _, a = Needleman_Wunsch(alg, s, scoring_matrix, -1)
      print(''.join(a))

In [23]:
def find_closest_pair(dist):
    #Поиск двух ближайших строк по расстоянию левенштейна
    min_distance = float('inf')
    closest_pair = ()
    for i in range(len(dist)):
        for j in range(i+1, len(dist)):
            if dist[i][j] < min_distance:
                min_distance = dist[i][j]
                closest_pair = (i, j)
    return closest_pair

In [24]:
def update_distance_matrix(distance_matrix, i, j, new_str, alignments, deletion_penalty, mismatch_penalty, match_score):
    #Пересчет расстояний для новой строки
    distance_matrix = np.delete(distance_matrix, j, axis=0)
    distance_matrix = np.delete(distance_matrix, j, axis=1)

    for line in range(len(alignments)):
        d = levenshtein_distance(alignments[line], new_str, deletion_penalty, match_score, mismatch_penalty)
        distance_matrix[line][i], distance_matrix[i][line] = d, d
    return distance_matrix

In [25]:
def calc_dist(strings, deletion_penalty, mismatch_penalty, match_score):
    #Вычисление всей матрицы расстояний
    n = len(strings)
    distance = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            dist = levenshtein_distance(strings[i], strings[j], deletion_penalty, match_score, mismatch_penalty)
            distance[i, j], distance[j, i] = dist, dist
    return distance

In [31]:
def multiple_alignment(strings, deletion_penalty, mismatch_penalty, match_score):
    ### с сохранением расстояний
    bases = ['A', 'T', 'G', 'C', '-']
    #матрица штрафов в духе blosam
    scoring_matrix = np.zeros((len(bases), len(bases)), dtype=int)
    for i in range(len(bases)):
        for j in range(len(bases)):
                if bases[i] == '-' or bases[j] == '-':
                    scoring_matrix[i][j] = deletion_penalty
                elif bases[i] == bases[j]:
                    scoring_matrix[i][j] = match_score
                else:
                    scoring_matrix[i][j] = mismatch_penalty

    alignments = strings.copy()
    #профили выравнивания
    profs = [first(seq) for seq in alignments]
    L_dist = calc_dist(alignments, deletion_penalty, mismatch_penalty, match_score)
    while len(alignments) > 1:
        #new pair
        i, j = find_closest_pair(L_dist)
        alg = Needleman_Wunsch(alignments[i], alignments[j], scoring_matrix, deletion_penalty)
        prof_i = get_prof(alg[0], alignments[i], i, profs)
        prof_j = get_prof(alg[1], alignments[j], j, profs)
        new_prof = prof_i + prof_j
        consensus = calculate_consensus(new_prof)

        #reinit
        del alignments[j]
        del profs[j]
        alignments[i] = consensus
        profs[i] = new_prof
        L_dist = update_distance_matrix(L_dist, i, j, consensus, alignments, deletion_penalty, mismatch_penalty, match_score)
    get_string(strings, alignments[0], scoring_matrix)

Пример использования:

In [36]:
deletion_penalty = -3
match_score = 2
mismatch_penalty = -1

In [33]:
strings = ["AGTACGCA", "TATGC", "TATGCA"]
multiple_alignment(strings, deletion_penalty, mismatch_penalty, match_score)

conc: AGTATGCA
AGTACGCA
--TATGC-
--TATGCA


In [34]:
seqs = ["ATTC", "GATTC", "AGATTC", "GATTC", "GGATTC", "CGATTCA"]
multiple_alignment(seqs, deletion_penalty, mismatch_penalty, match_score)

conc: AGATTCA
A--TTC-
-GATTC-
AGATTC-
-GATTC-
GGATTC-
CGATTCA


In [37]:
seqs = ["ATTCGGGGATA", "ATTG", "GATA", "CGAT", "ACTCA"]
multiple_alignment(seqs, deletion_penalty, mismatch_penalty, match_score)

conc: ATTCGGGGATA
ATTCGGGGATA
ATT-G------
----G---ATA
---CG---AT-
ACTC----A--
